In [2]:
!pip install ortools


     |████████████████████████████████| 11.8MB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 5.1MB/s eta 0:00:01
  Found existing installation: protobuf 3.12.2
    Uninstalling protobuf-3.12.2:
      Successfully uninstalled protobuf-3.12.2


In [15]:
from __future__ import print_function
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [4]:
#site 0 data
scenario1 = pd.read_csv("scenario1_0.csv")

In [6]:
scenario1.head()

,population,required_gdus,scenario_1_harvest_quantity,week_original_plant_date
0,p_56,1259,270,1
1,p_57,1259,389,1
2,p_58,1259,249,1
3,p_59,1356,100,1
4,p_60,1356,296,1


In [7]:
#sort scenario
scenario1.sort_values(by=['population'], inplace=True)

In [8]:
values = [] #
weights = [] #scenario 1 weight using GDU
week = []

In [10]:
for index, row in scenario1.iterrows():
    week.append(row["week_original_plant_date"])
    weights.append(row["required_gdus"])
    values.append(row["scenario_1_harvest_quantity"])

In [11]:
week = list(set(week))

In [16]:
data = {}
data['weights'] = weights
data['values'] = values
data['items'] = list(range(len(weights)))
data['num_items'] = len(weights)

In [20]:
num_bins = len(week)
data['bins'] = list(range(num_bins))
capacities = [7000 for i in range(1,num_bins)]
data['bin_capacities'] = capacities

In [18]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

In [24]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) <= 1)


In [ ]:
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i]
            for i in data['items']) <= data['bin_capacities'][j])

In [22]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        objective.SetCoefficient(x[(i, j)], data['values'][i])
objective.SetMaximization()

{'weights': [867,
  918,
  918,
  918,
  867,
  867,
  867,
  867,
  818,
  918,
  918,
  867,
  818,
  918,
  818,
  818,
  867,
  867,
  818,
  1054,
  1054,
  1141,
  900,
  954,
  954,
  1009,
  1009,
  954,
  954,
  1054,
  824,
  824,
  824,
  824,
  824,
  771,
  879,
  879,
  824,
  824,
  1141,
  879,
  879,
  771,
  879,
  771,
  824,
  879,
  879,
  824,
  824,
  1054,
  824,
  1009,
  879,
  879,
  879,
  824,
  824,
  879,
  879,
  879,
  1054,
  879,
  879,
  879,
  879,
  824,
  879,
  824,
  824,
  879,
  887,
  1054,
  833,
  1009,
  1009,
  900,
  1009,
  1009,
  1009,
  1009,
  954,
  1009,
  1054,
  900,
  900,
  1009,
  1009,
  887,
  942,
  887,
  887,
  887,
  942,
  1141,
  942,
  887,
  887,
  942,
  887,
  942,
  942,
  942,
  887,
  942,
  1054,
  942,
  942,
  833,
  887,
  942,
  1054,
  1054,
  1054,
  1141,
  1054,
  1054,
  1054,
  1054,
  1054,
  760,
  760,
  1141,
  760,
  825,
  825,
  825,
  825,
  889,
  876,
  876,
  876,
  876,
  1141,
  876,
  8